In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('fifa-transformation') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/13 10:18:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("basePath", "dataset/").parquet('dataset/*/*')
df.columns

['sofifa_id',
 'short_name',
 'overall',
 'age',
 'dob',
 'height_cm',
 'weight_kg',
 'nationality_name',
 'club_name',
 'league_name',
 'league_level',
 'potential',
 'value_eur',
 'wage_eur',
 'player_positions',
 'gender',
 'unique_id',
 'version']

In [4]:
print((df.count(), len(df.columns)))
df.show()

(144323, 18)
+---------+-------------+-------+---+-------------------+---------+---------+----------------+---------+-----------+------------+---------+---------+--------+----------------+------+-----------+-------+
|sofifa_id|   short_name|overall|age|                dob|height_cm|weight_kg|nationality_name|club_name|league_name|league_level|potential|value_eur|wage_eur|player_positions|gender|  unique_id|version|
+---------+-------------+-------+---+-------------------+---------+---------+----------------+---------+-----------+------------+---------+---------+--------+----------------+------+-----------+-------+
|   227246|    L. Bronze|     92| 29|1991-10-28 00:00:00|      171|       67|         England|  unknown|       null|        null|       92|     null|    null|          RB, CM|female|25769820906|     22|
|   227316|    W. Renard|     92| 30|1990-07-20 00:00:00|      187|       70|          France|  unknown|       null|        null|       92|     null|    null|              CB|

In [5]:
# TOTAL PLAYERS IN EACH FIFA BY GENDER

df.groupby('version', 'gender').count().orderBy('version', 'gender').show() 

+-------+------+-----+
|version|gender|count|
+-------+------+-----+
|     15|  male|16155|
|     16|female|  248|
|     16|  male|15623|
|     17|female|  299|
|     17|  male|17596|
|     18|female|  317|
|     18|  male|17954|
|     19|female|  299|
|     19|  male|18085|
|     20|female|  345|
|     20|  male|18483|
|     21|female|  345|
|     21|  male|18944|
|     22|female|  391|
|     22|  male|19239|
+-------+------+-----+



In [7]:
# Display the players with overall score > 90 for each fifa

df.groupby('version', 'gender', 'overall').count().filter("overall > 90").show()

+-------+------+-------+-----+
|version|gender|overall|count|
+-------+------+-------+-----+
|     18|  male|     91|    1|
|     20|female|     93|    1|
|     22|female|     92|    3|
|     20|female|     92|    2|
|     18|female|     91|    1|
|     20|  male|     94|    1|
|     22|  male|     92|    1|
|     22|female|     91|    1|
|     20|  male|     92|    1|
|     20|female|     91|    3|
|     18|female|     92|    1|
|     20|  male|     93|    1|
|     18|  male|     94|    1|
|     22|  male|     91|    5|
|     22|  male|     93|    1|
|     18|  male|     93|    1|
|     18|  male|     92|    3|
|     20|  male|     91|    3|
|     21|  male|     91|    4|
|     21|  male|     93|    1|
+-------+------+-------+-----+
only showing top 20 rows



In [4]:
# Create temp views for queries
df.createOrReplaceTempView("fifa")

In [29]:
# Best 20 fifa players per year

# TUNE version

df_top_players_by_year = spark.sql("""

SELECT version, overall, short_name
FROM fifa 
WHERE version = 17
ORDER BY overall DESC
LIMIT 11;
 """).show()


+-------+-------+-----------------+
|version|overall|       short_name|
+-------+-------+-----------------+
|     17|     94|Cristiano Ronaldo|
|     17|     93|         L. Messi|
|     17|     92|        L. Suárez|
|     17|     92|         M. Neuer|
|     17|     92|           Neymar|
|     17|     92|         C. Lloyd|
|     17|     91|       M. Rapinoe|
|     17|     90|   R. Lewandowski|
|     17|     90|           De Gea|
|     17|     90|          G. Bale|
|     17|     90|       J. Boateng|
|     17|     90|   Z. Ibrahimović|
|     17|     89|     Thiago Silva|
|     17|     89|     Sergio Ramos|
|     17|     89|            Marta|
|     17|     89|        L. Modrić|
|     17|     89|      C. Sinclair|
|     17|     89|      T. Courtois|
|     17|     89|        S. Agüero|
|     17|     89|          M. Özil|
+-------+-------+-----------------+
only showing top 20 rows



In [9]:
# TOP PLAYER IN EACH FIFA

df_top_players = spark.sql("""

SELECT version, short_name, overall
FROM fifa WHERE (version,overall) IN
( SELECT version, MAX(overall)
  FROM fifa
  GROUP BY version
)
ORDER BY version

""")


df_top_players.show()

+-------+-----------------+-------+
|version|       short_name|overall|
+-------+-----------------+-------+
|     15|         L. Messi|     93|
|     16|         L. Messi|     94|
|     17|Cristiano Ronaldo|     94|
|     18|Cristiano Ronaldo|     94|
|     19|Cristiano Ronaldo|     94|
|     19|         L. Messi|     94|
|     20|         L. Messi|     94|
|     21|       M. Rapinoe|     93|
|     21|         L. Messi|     93|
|     22|         L. Messi|     93|
+-------+-----------------+-------+



In [30]:
# Best potential fifa players per year

# TUNE version

df_top_players_by_year = spark.sql("""

SELECT version, potential, short_name
FROM fifa 
WHERE version = 17
ORDER BY potential DESC
 """).show()


+-------+---------+--------------------+
|version|potential|          short_name|
+-------+---------+--------------------+
|     17|       95|              Neymar|
|     17|       94|   Cristiano Ronaldo|
|     17|       94|A. Stolsmo Hegerberg|
|     17|       94|            P. Pogba|
|     17|       93|         J. Johnston|
|     17|       93|          V. Miedema|
|     17|       93|            L. Messi|
|     17|       93|    C. Graham Hansen|
|     17|       93|         T. Courtois|
|     17|       92|           L. Suárez|
|     17|       92|              De Gea|
|     17|       92|           P. Dybala|
|     17|       92|            C. Lloyd|
|     17|       92|        J. Rodríguez|
|     17|       92|            M. Neuer|
|     17|       92|        A. Griezmann|
|     17|       91|         D. Marozsán|
|     17|       91|          A. Kennedy|
|     17|       91|            J. Oblak|
|     17|       91|          M. Rapinoe|
+-------+---------+--------------------+
only showing top

In [37]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col, row_number

# Transformation to get, for a specific fifa version and player gender, the sum of the top 11 players overall score for each team
df_fifa_n = df.where((col('version') == 17) & (col('gender') == 'male'))

n = 11
df_best11_per_team = df_fifa_n.select(col('*'), row_number().over(window).alias('club_num')) \
    .where(col('club_num') <= n)

df_best11_per_team = df_best11_per_team.groupBy('version', 'club_name').sum('overall').orderBy('sum(overall)', ascending=False).show(10)

# df_best5_per_teams.select(['club_name', 'league_name', 'short_name', 'overall', 'gender', 'version']).show(500)

+-------+-------------------+------------+
|version|          club_name|sum(overall)|
+-------+-------------------+------------+
|     17|     Real Madrid CF|         967|
|     17|  FC Bayern München|         965|
|     17|       FC Barcelona|         964|
|     17|           Juventus|         945|
|     17|    Manchester City|         936|
|     17|            Chelsea|         935|
|     17|  Manchester United|         934|
|     17|            Arsenal|         933|
|     17|Paris Saint-Germain|         933|
|     17| Atlético de Madrid|         930|
+-------+-------------------+------------+
only showing top 10 rows



In [28]:
# SQL equivalent of previous block

# Top teams wrt to 'overall' score of their best 11 players for each fifa version and player gender

# TUNE version and gender

df_top_teams_by_year_and_gender = spark.sql("""
SELECT club_name, SUM(overall) total
FROM (
    SELECT version, gender, club_name, overall, ROW_NUMBER() OVER(PARTITION BY club_name ORDER BY overall DESC) rn
    FROM fifa
    WHERE version = 20 and gender = 'male' and club_name <> 'unknown'
) x 
WHERE rn <= 11
GROUP BY club_name
ORDER BY total DESC

 """).show(100)



+--------------------+-----+
|           club_name|total|
+--------------------+-----+
|        FC Barcelona|  968|
|      Real Madrid CF|  963|
|     Manchester City|  960|
|            Juventus|  952|
|           Liverpool|  945|
| Paris Saint-Germain|  945|
|   FC Bayern München|  944|
|   Tottenham Hotspur|  940|
|  Atlético de Madrid|  925|
|   Borussia Dortmund|  925|
|              Napoli|  924|
|               Inter|  921|
|             Arsenal|  917|
|   Manchester United|  914|
|             Chelsea|  911|
|               Lazio|  905|
|         Valencia CF|  903|
| Bayer 04 Leverkusen|  900|
| Real Betis Balompié|  898|
|            AC Milan|  896|
|                Ajax|  893|
|          RB Leipzig|  892|
|          Sevilla FC|  891|
|            FC Porto|  890|
|           AS Monaco|  889|
|          SL Benfica|  888|
|Athletic Club de ...|  886|
|                Roma|  884|
|             Everton|  883|
|  Olympique Lyonnais|  882|
|         Sporting CP|  881|
|     West Ham

In [31]:
# Best potential teams based on the 'potential' score of their best 11 players for each fifa version and player gender

# TUNE version and gender

df_top_teams_by_year_and_gender = spark.sql("""
SELECT club_name, SUM(potential) total
FROM (
    SELECT version, gender, club_name, potential, ROW_NUMBER() OVER(PARTITION BY club_name ORDER BY potential DESC) rn
    FROM fifa
    WHERE version = 20 and gender = 'male' and club_name <> 'unknown'
) x 
WHERE rn <= 11
GROUP BY club_name
ORDER BY total DESC

 """).show(100)



+--------------------+-----+
|           club_name|total|
+--------------------+-----+
|     Manchester City|  990|
|        FC Barcelona|  990|
|      Real Madrid CF|  984|
|            Juventus|  975|
|           Liverpool|  974|
| Paris Saint-Germain|  972|
|   Tottenham Hotspur|  970|
|   FC Bayern München|  969|
|  Atlético de Madrid|  965|
|   Manchester United|  958|
|   Borussia Dortmund|  956|
|              Napoli|  956|
|               Inter|  954|
|             Chelsea|  954|
|            AC Milan|  953|
|             Arsenal|  950|
|          RB Leipzig|  947|
|  Olympique Lyonnais|  943|
| Bayer 04 Leverkusen|  943|
|         Valencia CF|  940|
|                Roma|  939|
|          SL Benfica|  939|
|                Ajax|  936|
|               Lazio|  935|
|           AS Monaco|  935|
|Wolverhampton Wan...|  930|
|             Everton|  930|
|          LOSC Lille|  929|
|       Real Sociedad|  927|
|            FC Porto|  926|
|      Leicester City|  925|
| Real Betis B

In [35]:
# SQL equivalent of previous block

# Show the sum of the best N player values year by year

# TUNE N

df_top_teams_by_year_and_gender = spark.sql("""
SELECT version, SUM(value_eur) total
FROM (
    SELECT version, value_eur, ROW_NUMBER() OVER(PARTITION BY version ORDER BY value_eur DESC) rn
    FROM fifa
) x 
WHERE rn <= 100
GROUP BY version
ORDER BY total DESC

 """).show()



+-------+----------+
|version|     total|
+-------+----------+
|     21|7778500000|
|     22|7717000000|
|     19|5333000000|
|     20|5220500000|
|     18|4980000000|
|     17|4318500000|
|     16|3685000000|
|     15|2630500000|
+-------+----------+



In [ ]:
spark.stop()